In [1]:
import sklearn
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

In [16]:
db = pd.read_csv("database/LocTreino_Equipe_4.csv")
#db = db.to_numpy()
db

,lat,lon,rssi_1_1,rssi_1_2,rssi_1_3,rssi_2_1,rssi_2_2,rssi_2_3,rssi_3_1,rssi_3_2,rssi_3_3,delay_1,delay_2,delay_3,pontoId
0,-8.05438,-34.95408,-89.044286,-96.667143,-91.320000,-96.106667,-94.480000,-90.187143,-94.125000,-85.182000,-96.420000,5,4,2,7531
1,-8.05337,-34.94889,-91.970000,-89.548571,-88.555714,-82.520000,-80.445000,-89.910000,-88.916667,-91.602000,-97.255000,5,2,4,7748
2,-8.05390,-34.95955,-85.255714,-99.635714,-86.551429,-94.520000,-100.225000,-88.232857,-78.955000,-92.080000,-66.410000,6,7,1,7636
3,-8.05561,-34.95431,-90.025714,-96.200000,-91.204286,-98.160000,-91.935000,-92.671429,-78.636667,-68.340000,-92.780000,6,5,2,7035
4,-8.05562,-34.95405,-94.525714,-96.200000,-93.830000,-98.216667,-94.670000,-94.844286,-79.065000,-68.250000,-96.212500,6,5,2,7026
5,-8.05643,-34.95069,-91.218571,-92.157143,-91.926667,-88.840000,-84.935000,-82.520000,-91.623333,-89.130000,-95.730000,6,3,3,6808
6,-8.05007,-34.95474,-82.999429,-98.255357,-87.818571,-101.535000,-97.591250,-90.656905,-80.617222,-99.934667,-96.982778,3,5,3,8797
7,-8.05361,-34.94907,-87.700000,-89.400000,-90.595714,-74.385000,-76.877500,-76.907143,-106.260000,-83.630000,-96.720000,5,2,4,7693
8,-8.04900,-34.95697,-79.204286,-98.220000,-76.887143,-99.140000,-90.057500,-94.460000,-89.895000,-102.228667,-94.090000,4,6,3,9041
9,-8.05208,-34.94952,-95.900000,-90.420000,-99.160000,-70.750000,-90.725000,-84.915714,-91.413333,-96.142000,-95.567500,4,2,4,8088


In [22]:
X = db.drop("lat", axis = 1)
X = X.drop("lon", axis = 1)
X

,rssi_1_1,rssi_1_2,rssi_1_3,rssi_2_1,rssi_2_2,rssi_2_3,rssi_3_1,rssi_3_2,rssi_3_3,delay_1,delay_2,delay_3,pontoId
0,-89.044286,-96.667143,-91.320000,-96.106667,-94.480000,-90.187143,-94.125000,-85.182000,-96.420000,5,4,2,7531
1,-91.970000,-89.548571,-88.555714,-82.520000,-80.445000,-89.910000,-88.916667,-91.602000,-97.255000,5,2,4,7748
2,-85.255714,-99.635714,-86.551429,-94.520000,-100.225000,-88.232857,-78.955000,-92.080000,-66.410000,6,7,1,7636
3,-90.025714,-96.200000,-91.204286,-98.160000,-91.935000,-92.671429,-78.636667,-68.340000,-92.780000,6,5,2,7035
4,-94.525714,-96.200000,-93.830000,-98.216667,-94.670000,-94.844286,-79.065000,-68.250000,-96.212500,6,5,2,7026
5,-91.218571,-92.157143,-91.926667,-88.840000,-84.935000,-82.520000,-91.623333,-89.130000,-95.730000,6,3,3,6808
6,-82.999429,-98.255357,-87.818571,-101.535000,-97.591250,-90.656905,-80.617222,-99.934667,-96.982778,3,5,3,8797
7,-87.700000,-89.400000,-90.595714,-74.385000,-76.877500,-76.907143,-106.260000,-83.630000,-96.720000,5,2,4,7693
8,-79.204286,-98.220000,-76.887143,-99.140000,-90.057500,-94.460000,-89.895000,-102.228667,-94.090000,4,6,3,9041
9,-95.900000,-90.420000,-99.160000,-70.750000,-90.725000,-84.915714,-91.413333,-96.142000,-95.567500,4,2,4,8088


In [30]:
db["lat"]

0      -8.05438
1      -8.05337
2      -8.05390
3      -8.05561
4      -8.05562
5      -8.05643
6      -8.05007
7      -8.05361
8      -8.04900
9      -8.05208
10     -8.04825
11     -8.04863
12     -8.05069
13     -8.05375
14     -8.05560
15     -8.04848
16     -8.05512
17     -8.05662
18     -8.05398
19     -8.05111
20     -8.05546
21     -8.05447
22     -8.05511
23     -8.04926
24     -8.05463
25     -8.05305
26     -8.05827
27     -8.04812
28     -8.05153
29     -8.04680
         ...   
1470   -8.04906
1471   -8.05259
1472   -8.05099
1473   -8.05631
1474   -8.04994
1475   -8.05119
1476   -8.04898
1477   -8.04846
1478   -8.05210
1479   -8.04872
1480   -8.05542
1481   -8.05122
1482   -8.05329
1483   -8.05244
1484   -8.05039
1485   -8.05339
1486   -8.05212
1487   -8.05233
1488   -8.05160
1489   -8.05279
1490   -8.05369
1491   -8.05096
1492   -8.04988
1493   -8.05518
1494   -8.05648
1495   -8.05054
1496   -8.04978
1497   -8.05127
1498   -8.05338
1499   -8.05131
Name: lat, Length: 1500,